# Cursillo de minería de texto
## Centro Interdisciplinario para la Innovación en Salud
### Juan Zamora Osorio - Abril, 2016

# Parte I - Preprocesamiento

## Extracción de características
Consideremos el siguiente (mini) texto:

In [1]:
import nltk

txt_example = """Enron Corporation (former NYSE ticker symbol ENE) was an
American energy company based in Houston, Texas. Before its bankruptcy
in late 2001, Enron employed approximately 22,000[1] and was one of
the world’s leading electricity, natural gas, pulp and paper, and
communications companies, with claimed revenues of nearly $101 billion
in 2000."""

Al aplicar la función `word_tokenize`:

In [3]:
print nltk.word_tokenize(txt_example, language='english')

['Enron', 'Corporation', '(', 'former', 'NYSE', 'ticker', 'symbol', 'ENE', ')', 'was', 'an', 'American', 'energy', 'company', 'based', 'in', 'Houston', ',', 'Texas', '.', 'Before', 'its', 'bankruptcy', 'in', 'late', '2001', ',', 'Enron', 'employed', 'approximately', '22,000', '[', '1', ']', 'and', 'was', 'one', 'of', 'the', 'world\xe2\x80\x99s', 'leading', 'electricity', ',', 'natural', 'gas', ',', 'pulp', 'and', 'paper', ',', 'and', 'communications', 'companies', ',', 'with', 'claimed', 'revenues', 'of', 'nearly', '$', '101', 'billion', 'in', '2000', '.']


Otra opción consiste en usar [expresiones regulares](https://es.wikipedia.org/wiki/Expresi%C3%B3n_regular)

In [4]:
import re

pat = r"""(?ux)
(?:[^\W\d_]\.)+                   # Abreviaciones
| [^\W\d_]+(?:-[^\W\d_])*(?:'s)?  # Palabras con guión (compuestas)
| \d{4}                           # Año
| \d{1,3}(?:,\d{3})*              # Numero
| \$\d+(?:\.\d{2})?               # Moneda
| \d{1,3}(?:\.\d+)?\s%            # Porcentaje
| \.\.\.                          # Puntos suspensivos (elipsis)
| [.,:"'?!():-__`/]               # Otros simbolos
"""

print nltk.regexp_tokenize(txt_example, pat)

['Enron', 'Corporation', '(', 'former', 'NYSE', 'ticker', 'symbol', 'ENE', ')', 'was', 'an', 'American', 'energy', 'company', 'based', 'in', 'Houston', ',', 'Texas', '.', 'Before', 'its', 'bankruptcy', 'in', 'late', '2001', ',', 'Enron', 'employed', 'approximately', '22,000', '[', '1', ']', 'and', 'was', 'one', 'of', 'the', 'world\xe2', 's', 'leading', 'electricity', ',', 'natural', 'gas', ',', 'pulp', 'and', 'paper', ',', 'and', 'communications', 'companies', ',', 'with', 'claimed', 'revenues', 'of', 'nearly', '$101', 'billion', 'in', '2000', '.']


## Categorías de términos y reducción de dimensionalidad

### Lematización (Stemming):

In [5]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
print "defines -->",stemmer.stem("defines")
print "define -->",stemmer.stem("define")
print "defining -->",stemmer.stem("defining")

defines --> defin
define --> defin
defining --> defin


__Pregunta:__ ¿Como lematizar todos los términos de la lista `items`?

### Part-of-speech tagging:

NN noun, VB verb, JJ adjective, RB adverb ...(revisar salida de la instrucción `nltk.help.upenn_tagset()`)

Se encontrará más detalle en el [libro en línea del libro de _NLTK_](http://www.nltk.org/book/ch05.html)

In [7]:
print nltk.pos_tag( nltk.word_tokenize(txt_example, language='english') )

[('Enron', 'NNP'), ('Corporation', 'NNP'), ('(', '('), ('former', 'JJ'), ('NYSE', 'NNP'), ('ticker', 'NN'), ('symbol', 'NN'), ('ENE', 'NNP'), (')', ')'), ('was', 'VBD'), ('an', 'DT'), ('American', 'JJ'), ('energy', 'NN'), ('company', 'NN'), ('based', 'VBN'), ('in', 'IN'), ('Houston', 'NNP'), (',', ','), ('Texas', 'NNP'), ('.', '.'), ('Before', 'IN'), ('its', 'PRP$'), ('bankruptcy', 'NN'), ('in', 'IN'), ('late', 'JJ'), ('2001', 'CD'), (',', ','), ('Enron', 'NNP'), ('employed', 'VBD'), ('approximately', 'RB'), ('22,000', 'CD'), ('[', 'JJ'), ('1', 'CD'), (']', 'NN'), ('and', 'CC'), ('was', 'VBD'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('world\xe2\x80\x99s', 'NN'), ('leading', 'VBG'), ('electricity', 'NN'), (',', ','), ('natural', 'JJ'), ('gas', 'NN'), (',', ','), ('pulp', 'NN'), ('and', 'CC'), ('paper', 'NN'), (',', ','), ('and', 'CC'), ('communications', 'NNS'), ('companies', 'NNS'), (',', ','), ('with', 'IN'), ('claimed', 'JJ'), ('revenues', 'NNS'), ('of', 'IN'), ('nearly', 'RB'),

__Comentario:__ Pruebe la última instrucción convirtiendo previamente el texto a minúscula y revise la salida.

#### Que sucede si ensamblan sustantivos consecutivos...

In [8]:
tagged = nltk.pos_tag( nltk.word_tokenize(txt_example, language='english') )
phrases, phrase = [], ""
for (word, tag) in tagged:
    if tag[:2] == "NN":
        if phrase == "":
            phrase = word
        else:
            phrase += " " + word
    elif phrase != "":
        phrases.append(phrase.lower())
        phrase = ""
print phrases

['enron corporation', 'nyse ticker symbol ene', 'energy company', 'houston', 'texas', 'bankruptcy', 'enron', ']', 'world\xe2\x80\x99s', 'electricity', 'gas', 'pulp', 'paper', 'communications companies', 'revenues']


## De documentos a términos y finalmente a vectores en $\mathbb{R}$

* Se debe primero generar un vocabulario $\mathcal{V}$ con todos los términos considerados como descriptores de cada documento
* Identificar para cada documento la cantidad de ocurrencias a partir de los términos del vocabulario
* Construir un vector de largo $|\mathcal{V}|$ que represente a cada documento usando las ocurrencias de cada término en él

## Procesamiento de una colección de documentos

In [9]:
import csv
import string

In [10]:
#lines = csv.reader(open("coleccion.csv"), delimiter=',', quotechar='"')
lines = csv.reader(open("20ng_labeled.csv"), delimiter=';', quotechar="'")

header = []
papers = []

for row in lines:
    line = [unicode(cell, 'utf-8') for cell in row]
    if not header:
        header = line
        continue
    papers.append(dict(zip(header, line)))

print dict(zip(header, line))

{u'Content': u'You forgot the smiley-face.  I cant believe this is what they turn out at Berkeley.  Tell me youre an aberration.', u'Label': u'2'}


Para entender que hace la última línea, ver el ejemplo a continuación:

In [11]:
print zip([1,2,3,4],['a','b','c','d'])
print dict(zip([1,2,3,4],['a','b','c','d']))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]
{1: 'a', 2: 'b', 3: 'c', 4: 'd'}


Ver por ejemplo como queda estructurado uno de los documentos de la colección:

In [12]:
print papers[0].keys()

[u'Content', u'Label']


### Selección de características mediante eliminación de Stopwords

__(Volviendo a la colección original)__ Ahora se extraeran los términos (palabras) que componen cada _abstract_ usando la función `word_tokenize`:

In [13]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

collection_words = {}

for paper in papers:
    paper_words = nltk.word_tokenize(paper['Content'].lower())
    words = {}    
    for w in paper_words:
        if w not in stopwords and len(w) > 2:
            words[w] = words.get(w, 0) + 1    
    paper.update({'words': words})
    for term in words:
        collection_words[term] = collection_words.get(term, 0) + 1.0

__Pregunta:__ ¿Como realizar esto usando expresiones regulares?

__Pregunta:__ ¿Que contiene el diccionario asociado a la llave `words` para cada paper?

__Pregunta:__ ¿Que contiene el diccionario `collection_words`?

In [14]:
from scipy import sparse as sp
import numpy as np

V = [w for w in collection_words.keys()]
# Se crea la matriz de documentos vs terminos
N = len(papers)
d = len(V)
data = sp.lil_matrix((N, d))
labels = np.zeros(N,)

Luego se generan los vectores para los documentos (uno por cada fila de la matriz). Este paso tarda un tiempo dependiendo de la cantidad y tamaño de los documentos:

In [15]:
for docid in range( len(papers) ):
    paper = papers[docid]
    labels[docid] = paper['Label']
    for w in paper['words']:
        data[docid, V.index(w)] = paper['words'][w] * np.log(N/collection_words[w])

Para usos posteriores quizás sea conveniente serializar la matriz ya construída, almacenarla en el disco y así acelerar su carga futura:

`
import pickle
pickle.dump( data, open( "doc_term_matrix.p", "wb" ) )
`

Para recuperar la matriz a partir del archivo serializado se debe ejecutar:

`data = pickle.load( open( "doc_term_matrix.p", "rb" ) )`

# Parte II - Análisis Automático sobre colecciones documentales

__Primero__ una versión alternativa ~~y menos controlada~~ para la construcción del vocabulario y de la matriz de documentos vs términos. Para más detalle revisar el enlace en [Scikit-learn](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [20]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

newsgroups_train = fetch_20newsgroups(subset='train',categories=['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med'], remove=('headers', 'footers', 'quotes'))
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(newsgroups_train.data)
tfidf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
print "Se contabilizaron",X_train_tfidf.shape[0],"con un vocabulario de",X_train_tfidf.shape[1],"términos."

Se contabilizaron 2257 con un vocabulario de 28865 términos.


### Usando un clasificador Naive Bayes Multinomial y SVM en Scikit-learn

Entrenamiento del clasificador NBM:

In [21]:
from sklearn.naive_bayes import MultinomialNB
clf_sk = MultinomialNB().fit(X_train_tfidf, newsgroups_train.target)

Entrenamiento de la SVM:

In [22]:
from sklearn.linear_model import SGDClassifier

svm_clf_sk = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42).fit(X_train_tfidf, newsgroups_train.target)

### Generando el dataset de prueba:

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
newsgroups_test = fetch_20newsgroups(subset='test',categories=['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med'], remove=('headers', 'footers', 'quotes'))
X_new_counts = count_vect.transform(newsgroups_test.data)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted_clf_sk = clf_sk.predict(X_new_tfidf)
predicted_svm_clf_sk = svm_clf_sk.predict(X_new_tfidf)

### Evaluación del desempeño de ambos clasificadores:

In [24]:
print "[Precision] NBM:{0:.4f} SVM:{1:.4f}".format(np.mean(predicted_clf_sk == newsgroups_test.target),np.mean(predicted_svm_clf_sk == newsgroups_test.target))

[Precision] NBM:0.6691 SVM:0.7956


### Generando dataset de prueba manualmente

Se realiza el mismo procedimiento anterior (desde cargar el archivo CSV), usando el vocabulario ya construído.

In [25]:
lines = csv.reader(open("20ng_labeled_test.csv"), delimiter=';', quotechar="'")

header = []
papers_test = []

for row in lines:
    line = [unicode(cell, 'utf-8') for cell in row]
    if not header:
        header = line
        continue
    papers_test.append(dict(zip(header, line)))

print dict(zip(header, line))

{u'Content': u'Yes, it looks like very good indeed.   Nope.', u'Label': u'1'}


In [26]:
for paper in papers_test:
    paper_words = nltk.word_tokenize(paper['Content'].lower())
    words = {}    
    for w in paper_words:
        if w in V and w not in stopwords and len(w) > 2:
            words[w] = words.get(w, 0) + 1    
    paper.update({'words': words})

In [27]:
N_test = len(papers_test)
data_test = sp.lil_matrix((N_test, d))
labels_test = np.zeros(N_test,)

In [28]:
for docid in range( len(papers_test) ):
    paper = papers_test[docid]
    labels_test[docid] = paper['Label']
    for w in paper['words']:
        data_test[docid, V.index(w)] = paper['words'][w] * np.log(N/collection_words[w])

pickle.dump( data_test, open( "doc_term_matrix_test.p", "wb" ) )        

### Usando un clasificador Naive Bayes Multinomial y SVM entrenados manualmente

In [29]:
clf_jz = MultinomialNB().fit(data, labels)
predicted_clf_jz = clf_jz.predict(data_test)


svm_clf_jz = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42).fit(data, labels)
predicted_svm_clf_jz = svm_clf_jz.predict(data_test)

### Evaluación del desempeño de ambos clasificadores:

In [30]:
print "[Precision] NBM:{0:.4f} SVM:{1:.4f}".format(np.mean(predicted_clf_jz == labels_test), np.mean(predicted_svm_clf_jz == labels_test))

[Precision] NBM:0.8336 SVM:0.7583


## Latent Semantic Analysis

Se realiza la descomposición SVD sobre la matriz de documentos vs términos:

In [31]:
from scipy.sparse.linalg import svds
U,S,Vt = svds(data, k=300)

In [ ]:
print "U:{0} S:{1} Vt:{2}".format( U.shape,np.diag(S).shape,Vt.shape )

U:(2257, 300) S:(300, 300) Vt:(300, 33084)


In [ ]:
from numpy.linalg import svd as SVDn
Up,Sp,Vp = SVDn(data.todense())

Aproximación de rango $k$ de la matriz:

In [ ]:
k = 100
rk_data = np.dot(U[:,:k], np.diag(S)[:k,:k]).dot(Vt[:k,:])

In [ ]:
from numpy.linalg import norm
print rk_data.shape
#norm(data.todense()-rk_data)

# Referencias

## Librerías (API)

[The Natural Language Toolkit](http://www.nltk.org/)

[Machine Learning in Python](http://scikit-learn.org/)

## Libros

[Introduction to Information Retrieval (_gratis_)](http://nlp.stanford.edu/IR-book/)